In [6]:
import numpy as np
import pandas as pd

In [7]:
mean = [0 for _ in range(5)]
cov = np.eye(5)
normal = pd.DataFrame(np.random.multivariate_normal(mean=mean, cov=cov, size = 500))

In [8]:
normal

,0,1,2,3,4
0,-0.136886,0.940475,-1.264041,-0.152750,0.621957
1,0.004551,1.106729,0.410703,-0.447793,-0.182955
2,-0.999254,-0.064771,0.672332,0.904433,-0.236411
3,-1.663768,-1.332106,-0.494719,-1.008706,1.051874
4,1.869622,1.278551,-1.251339,-0.754981,0.603141
...,...,...,...,...,...
495,0.791640,-0.543855,-1.270196,-0.159814,-0.289962
496,0.808149,-0.120416,-1.439854,-0.014542,2.345953
497,-0.833731,-0.144056,0.552080,-1.510491,0.261370
498,-0.781796,-1.854859,0.890415,0.876963,1.349611
